In [1]:
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

import numpy as np

from multiprocessing import Process, Queue, Manager

import plotly.graph_objects as go
from plotly.subplots import make_subplots
from Mapping.BoundingBox import BoundingBox

import time

In [2]:
points = [[(1,0,0),(2,1,1)], []]
points[0] += [(3,0,0)]

In [3]:
manager = Manager()
points = manager.list(points)

In [4]:
if len(points[0]) == 0:
    x,y,z = [],[],[]
else: x,y,z = zip(*points[0])

app = dash.Dash("Point", update_title=None)
fig1 = go.FigureWidget(
    data=[go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=2,
            color=z,
            colorscale='blackbody')
    )])
fig2 = go.FigureWidget(
    data=[go.Scatter3d(
        x=[], y=[], z=[],
        mode='markers',
        marker=dict(
            size=2,
            color=z,
            colorscale='blackbody')
    )])

fig3 = go.FigureWidget()

fig3.add_scatter3d(
    mode='markers',
    marker=dict(
        size=2,
        color="black"
    ),
    showlegend=True,
    name="expl. points"
)

fig3.add_mesh3d(
    i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
    j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
    k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
    opacity=1,
    color='#DC143C',
    flatshading = True,
    showlegend=True,
    name="robot"
)

fig3.update_layout(
    title="explored points",
    xaxis_title="y",
    yaxis_title="x" #use same system as holoocean
)

app.layout =  html.Div([
    html.Div([
        dcc.Graph(
        id="graph1",
        figure=fig1
    )
    ]),
    html.Div([
        dcc.Graph(
        id="graph2",
        figure=fig2, 
    ), 
    ]),
    dcc.Interval(
        id='interval-component',
        interval=1*1000, 
        n_intervals=0
    )
])

@app.callback(
    Output("graph1", "figure"),
    [Input('interval-component', 'n_intervals')],
    prevent_initial_call=False
)
def update_graph1(intervals):
    global points
    if len(points[0]) == 0:
        x, y, z = [],[],[]
    else: x,y,z = zip(*points[0])
    return go.Figure(
    data=[go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=2,
            color=z,
            colorscale='blackbody')
    )])

@app.callback(
    Output("graph2", "figure"),
    [Input('interval-component', 'n_intervals')],
    prevent_initial_call=False
)
def update_graph2(intervals):
    global points
    if len(points[1]) == 0:
        x, y, z = [],[],[]
    else: x,y,z = zip(*points[1])
    return go.Figure(
    data=[go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        marker=dict(
            size=2,
            color=z,
            colorscale='blackbody')
    )])

In [5]:
%%capture
def tmp(host, debug, app):
    app.run_server(host=host, debug=debug, threaded=True)
    
p = Process(target=app.run_server, kwargs={'host': '0.0.0.0', 'debug': False})
p.start()

Dash is running on http://0.0.0.0:8050/



In [7]:
tmp = points

In [10]:
tmp[0] += [(1,20,3)]

In [11]:
tmp[0]

[(1, 0, 0), (2, 1, 1), (3, 0, 0), (1, 20, 3)]

In [6]:
fig = app.layout.children[2].children[0].figure

In [7]:
print(fig)

FigureWidget({
    'data': [{'marker': {'color': 'black', 'size': 2},
              'mode': 'markers',
              'name': 'expl. points',
              'showlegend': True,
              'type': 'scatter3d',
              'uid': '831c6698-e39c-4d00-a7a4-b4241d83192b'},
             {'color': '#DC143C',
              'flatshading': True,
              'i': [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
              'j': [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
              'k': [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
              'name': 'robot',
              'opacity': 1,
              'showlegend': True,
              'type': 'mesh3d',
              'uid': 'e1628d55-0aa6-4c9a-b7c7-eb2a72222812'}],
    'layout': {'template': '...',
               'title': {'text': 'explored points'},
               'xaxis': {'title': {'text': 'y'}},
               'yaxis': {'title': {'text': 'x'}}}
})


In [14]:
points[0] += [(3,4,3)] 

In [7]:
points[1] += [(3,300,3)] 

In [ ]:
print(points[0])

In [ ]:
print(points)

In [ ]:
def update_fig(points, target, rob_pos, rob_width=.25, rob_height=.25, rob_depth=.25):
    x,y,z = zip(*points)
    fig = go.Figure(
        data=[go.Scatter3d(
            x=x, y=y, z=z,
            mode='markers',
            marker=dict(
                size=2,
                color="black"
            ),
            showlegend=False,
        )],
    )
    bbox = BoundingBox(
        (rob_pos[0]-rob_width, rob_pos[1]-rob_height, rob_pos[2]-rob_depth),
        (rob_pos[0]+rob_width, rob_pos[1]+rob_height, rob_pos[2]+rob_depth),
    ).get_bounding_box()
    fig.add_trace(go.Scatter3d(
        x=[target[0]], y=[target[1]], z=[target[2]],
        showlegend=False,
        marker=dict(
            size=5,
            color="blue"
        )
    ))
    fig.add_trace(go.Mesh3d(
            # 8 vertices of a cube
            x=[bbox[0][0], bbox[0][0], bbox[1][0], bbox[1][0], bbox[0][0], bbox[0][0], bbox[1][0], bbox[1][0]],
            y=[bbox[0][1], bbox[1][1], bbox[1][1], bbox[0][1], bbox[0][1], bbox[1][1], bbox[1][1], bbox[0][1]],
            z=[bbox[0][2], bbox[0][2], bbox[0][2], bbox[0][2], bbox[1][2], bbox[1][2], bbox[1][2], bbox[1][2]],
            i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
            j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
            k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
            opacity=1,
            color='#DC143C',
            flatshading = True,
            showlegend=False
        ))
    
    fig.update_layout(
        title="Position"
    )
    
    return fig

In [ ]:
points[0] += [(0,1,0)]
fig = update_fig(points[0], (0,2,2), (0,1,1))

In [ ]:
fig.show()

In [ ]:
points[0] += [(0,-1,-2)]
with fig.batch_update():
    x,y,z = zip(*points[0])
    fig.data[0].x = x
    fig.data[0].y = y
    fig.data[0].z = z

In [ ]:
print(fig.data)

In [6]:
def init_fig():
    fig = go.FigureWidget()
    fig.add_scatter3d(
        mode='markers',
        marker=dict(
            size=2,
            color="black"
        ),
        showlegend=False,
    )
    fig.add_scatter3d(
        showlegend=False,
        marker=dict(
            size=5,
            color="blue"
        )
    )
    fig.add_mesh3d(
        i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
        j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
        k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
        opacity=1,
        color='#DC143C',
        flatshading = True,
        showlegend=False
    )
    
    return fig

In [7]:
fig = init_fig()
display(fig)

FigureWidget({
    'data': [{'marker': {'color': 'black', 'size': 2},
              'mode': 'markers',
       …

In [8]:
def update_fig2(fig, points, target, rob_pos, rob_width=.25, rob_height=.25, rob_depth=.25):
    x,y,z = zip(*points)
    with fig.batch_update():
        fig.data[0].x = x
        fig.data[0].y = y
        fig.data[0].z = z
        
        fig.data[1].x = [target[0]]
        fig.data[1].y = [target[1]]
        fig.data[1].z = [target[2]]
        
        bbox = BoundingBox(
            (rob_pos[0]-rob_width, rob_pos[1]-rob_height, rob_pos[2]-rob_depth),
            (rob_pos[0]+rob_width, rob_pos[1]+rob_height, rob_pos[2]+rob_depth),
        ).get_bounding_box()
        
        fig.data[2].x = [bbox[0][0], bbox[0][0], bbox[1][0], bbox[1][0], bbox[0][0], bbox[0][0], bbox[1][0], bbox[1][0]]
        fig.data[2].y = [bbox[0][1], bbox[1][1], bbox[1][1], bbox[0][1], bbox[0][1], bbox[1][1], bbox[1][1], bbox[0][1]]
        fig.data[2].z = [bbox[0][2], bbox[0][2], bbox[0][2], bbox[0][2], bbox[1][2], bbox[1][2], bbox[1][2], bbox[1][2]]

In [9]:
update_fig2(fig, points[0], (0,5,2), (0,1,1))

In [ ]:
points[0] += [(4,2,2)]

In [13]:
update_fig2(fig, points[0], (0,5,2), (0,30,1))

In [ ]:
iteration_number = 0
while iteration_number < 100:
#    print(f"iteration {iteration_number + 1}")
    print("\rIteration {}".format(iteration_number + 1), end="")
    iteration_number += 1

In [ ]:
fig.update_layout(
    scene = dict(
        yaxis = dict(range=[-10,10],),
    )
)

In [ ]:
ipy = None
try:
    ipy = get_ipython().__class__.__name__ is not None
except:
    ipy = False

In [ ]:
ipy

In [ ]:
t = [1,2,3]

In [ ]:
manager = Manager()
X = manager.list(t)

In [ ]:
print(X)

In [ ]:
t.append(45)

In [ ]:
t

In [ ]:
print(X)

In [ ]:
X.append(45)

In [ ]:
print(X)

In [ ]:
def tmp(shared):
    while True:
        print(shared)
        time.sleep(1)
p = Process(
    target=tmp, args=[X]
)
p.start()

In [ ]:
X.append(12)

In [6]:
print(manager.list([None]*3))

[None, None, None]
